In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.gridspec as gridspec
from matplotlib.ticker import AutoMinorLocator

import sys, os
# This is not super pretty, but I think this is the best way to import stuff from ../../../util?
CODE_ROOT = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
if CODE_ROOT not in sys.path:
    sys.path.insert(1, CODE_ROOT)

from util.sim_data_helpers import get_cosmo_parameters
from util.cosmo_helpers import t_matter_lambda

## Get Cosmological parameters of boxes

In [ ]:
gps_to_plot = [i for i in range(50)]
base_path = "/vera/u/jerbo/my_ptmp/L25n256_suite"
gp_paths = [base_path+f"/gridpoint{i}/" for i in gps_to_plot]

params = []
for current_gp in gp_paths:
    Omega0, OmegaBaryon, OmegaLambda, HubbleParam = get_cosmo_parameters(current_gp)
    params.append([Omega0, OmegaBaryon, OmegaLambda, HubbleParam])

params = np.array(params)


## Calculate time as a function of scale factor

In [ ]:
a = np.linspace(0, 1, 1000)

ts = []
for Omega0, OmegaBaryon, OmegaLambda, HubbleParam in params:
    t = t_matter_lambda(a, Omega0, OmegaLambda, HubbleParam)
    t = t/(3600*24*365) * 1e-9  # convert to Gyrs
    ts.append(t)

## Create reference line

In [ ]:
Omega0_ref = 0.3089
OmegaBaryon_ref = 0.0486
OmegaLambda_ref = 0.6911
Hubble_Param_ref = 0.6774
params_ref = [Omega0_ref, OmegaBaryon_ref, OmegaLambda_ref, Hubble_Param_ref]

t_ref = t_matter_lambda(a, Omega0_ref, OmegaLambda_ref, Hubble_Param_ref)
t_ref = t_ref/(3600*24*365) * 1e-9  # convert to Gyrs


## Creat Plot

In [ ]:
color_param_index = 3
color_param_name = {0: r'$\Omega_m$',
                    1: r'$\Omega_b$',
                    2: r'$\Omega_\Lambda$',
                    3: r'$h$'}
Color_parameter = params[:, color_param_index]

fig, ax = plt.subplots()

ax.set_xticks([])
ax.set_yticks([])
for spine in ax.spines.values():
    spine.set_visible(False)

gs = gridspec.GridSpec(1, 2, width_ratios=[1, 0.05], wspace=0.1)

ax_lines = fig.add_subplot(gs[0, 0])
# ax_points = fig.add_subplot(gs[1, 1], sharey=ax_lines)
cbar_ax = fig.add_subplot(gs[0, 1])

norm = colors.Normalize(vmin=min(Color_parameter), vmax=max(Color_parameter))
colormap = cm.viridis  # Choose any colormap: viridis, plasma, inferno, etc.
scalar_map = cm.ScalarMappable(norm=norm, cmap=colormap)

for i, t in enumerate(ts):
    color = scalar_map.to_rgba(Color_parameter[i])
    ax_lines.plot(a, t, color=color)

ax_lines.plot(a, t_ref, color="black", linestyle="--", linewidth=3, label="reference")

ax_lines.plot([1/(2+1), 1/(2+1)], [0, 10], linestyle=":", alpha=0.7, color="black")
ax_lines.text(1/(2+1)-0.02, 7.5, "z=2", color="black", alpha=0.7)

ax_lines.set_xlim([1, -0.05])
ax_lines.yaxis.set_minor_locator(AutoMinorLocator())
ax_lines.set_ylabel("time [Gyrs]")
ax_lines.set_xlabel("scale factor")

cbar = plt.colorbar(scalar_map, cax=cbar_ax)
cbar.set_label(color_param_name[color_param_index])
leg = ax_lines.legend()
for line in leg.get_lines():
    line.set_linewidth(2)

plt.savefig("plots/time_in_boxes_visualization.pdf", format="PDF")
plt.show()

In [ ]:
scatter_param_index = 3
color_param_index = 2
size_param_index = 0
color_param_name = {0: r'$\Omega_m$',
                    1: r'$\Omega_b$',
                    2: r'$\Omega_\Lambda$',
                    3: r'$h$'}
Color_parameter = params[:, color_param_index]
scatter_parameter = params[:, scatter_param_index]
size_paramater = params[:, size_param_index]

fig, ax = plt.subplots()

ax.set_xticks([])
ax.set_yticks([])
for spine in ax.spines.values():
    spine.set_visible(False)

gs = gridspec.GridSpec(1, 2, width_ratios=[1, 0.05], wspace=0.1)

ax_dots = fig.add_subplot(gs[0, 0])
# ax_points = fig.add_subplot(gs[1, 1], sharey=ax_lines)
cbar_ax = fig.add_subplot(gs[0, 1])

norm = colors.Normalize(vmin=min(Color_parameter), vmax=max(Color_parameter))
colormap = cm.viridis  # Choose any colormap: viridis, plasma, inferno, etc.
scalar_map = cm.ScalarMappable(norm=norm, cmap=colormap)

for i, par in enumerate(scatter_parameter):
    color = scalar_map.to_rgba(Color_parameter[i])
    size = size_paramater[i]*200
    ax_dots.scatter(par, ts[i][-1], color=color, s=size)

ax_dots.scatter(params_ref[scatter_param_index], t_ref[-1], marker="*", s=100, color=scalar_map.to_rgba(params_ref[color_param_index]), label="reference")

############################

# Creating Visual Guide for Sizes
sizes = [min(size_paramater), (max(size_paramater)+min(size_paramater))/2, max(size_paramater)]
outline_x = max(scatter_parameter)*0.95 # Adjust as needed based on your plot scale
outlines_y = [19, 18, 17]  # Starting Y value, adjust based on your data range

for y_pos, size in zip(outlines_y, sizes):
    ax_dots.scatter(outline_x,
                y_pos,
                s=size*200,
                marker='o',
                facecolors='none',
                edgecolors='black',
                linewidths=1.5,
                alpha=0.7,
                zorder=10)

    ax_dots.text(outline_x * 1.01, y_pos*0.998, fr"{color_param_name[size_param_index]}={size:.2f}", va='center', ha='left', fontsize=9, color="black", alpha=0.7)


############################

ax_dots.set_ylabel("Age of the Universe [Gyrs]")
ax_dots.set_xlabel(f"{color_param_name[scatter_param_index]}")

cbar = plt.colorbar(scalar_map, cax=cbar_ax)
cbar.set_label(color_param_name[color_param_index])

ax_dots.yaxis.set_minor_locator(AutoMinorLocator())
ax_dots.legend()

plt.savefig("plots/age_of_universis_visualization.pdf", format="PDF")
plt.show()